### get accuracy of sivqa evaluation

idefics2 model

In [5]:
! pip install scikit-learn

  Using cached scikit_learn-1.5.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.0 MB/s eta 0:00:00
  Using cached scipy-1.13.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.5.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 170.8 MB/s eta 0:00:0000:010:01
Using cached scipy-1.13.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.2 MB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)


In [1]:
import utils, sivqa_utils
import os 
import json
from sklearn.metrics import accuracy_score
import re
import random

In [2]:
# result_dir = "/scratch3/wenyan/data/foodie/results"
result_dir = "/scratch/project/dd-23-107/wenyan/data/foodie/results"

In [3]:
# sivqa = sivqa_utils.read_sivqa("/scratch3/wenyan/data/foodie")
sivqa = sivqa_utils.read_sivqa("/scratch/project/dd-23-107/wenyan/data/foodie/")

### eval mantis accuracy

In [3]:
data = []
with open(os.path.join(result_dir, "sivqa_mantis_prompt3.jsonl"), "r", encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

In [29]:

ans2idx = {
        "A":"0",
        "B":"1",
        "C":"2",
        "D":"3"
        }

def parse_res(res):
    ans_str = res["response"][0].split("\nAssistant:")[-1].strip()
    ans_letter = re.findall(r'[A-Z]', ans_str)
    if not ans_letter or len(ans_letter) == 0:
        print("can not parse ans for res: ", res)
        return random.choice(["0", "1", "2", "3"])
    else:
        ans = ans_letter[0].upper()
        if ans not in ans2idx:
            print("can not parse ans for res: ", res)
            return random.choice(["0", "1", "2", "3"])
        else:
            return ans2idx[ans]


In [31]:
data[0]

{'response': ['User: 图片中的食物通常属于哪个菜系? 这是选项: （A) 京菜\n（B) 徽菜\n（C) 新疆菜\n（D) 桂菜\n 请根据上图从所提供的选项中选择一个正确答案。 \nAssistant: 我选择（ \nAssistant:B'],
 'qid': 'vqa-0'}

In [30]:
gts = [s["answer"] for s in sivqa]
answers = [parse_res(d) for d in data]
accuracy = accuracy_score(gts, answers)
print(accuracy)

0.40234375


get qwen accuracy

In [28]:
data = []
with open(os.path.join(result_dir, "sivqa_qwen_prompt2.jsonl"), "r", encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

In [29]:
data[0]["response"]#.split("为")[-1].strip()

'Picture 1:<img>/scratch3/wenyan/data/foodie/14456664_all_202404292352223293/217_IMG_3854.jpeg</img>\n用户：你是一个智能助手，现在请看图回答以下选择题：图片中的食物通常属于哪个菜系? 选项有: （A) 京菜\n（B) 徽菜\n（C) 新疆菜\n（D) 桂菜\n智能助手：我从所提供的选项中选择一个正确答案，为（C) 新疆菜。<|endoftext|>'

In [30]:

ans2idx = {
        "A":"0",
        "B":"1",
        "C":"2",
        "D":"3"
        }

def parse_res(res, template=0):
    if template == 0 or template ==1 or template == 3:
        ans_str = res["response"].split("选择一个正确答案，")[1].strip()
    ans_letter = re.findall(r'([A-Z])', ans_str)
    if not ans_letter or len(ans_letter) == 0:
        print("can not parse ans for res: ", res)
        return random.choice(["0", "1", "2", "3"])
    else:
        ans = ans_letter[0].upper()
        if ans not in ans2idx:
            print("can not parse ans for res: ", res)
            return random.choice(["0", "1", "2", "3"])
        else:
            return ans2idx[ans]


In [31]:
parse_res(data[0])

'2'

In [11]:
def get_accuracy(data, sivqa, template=0):
    gts = [s["answer"] for s in sivqa]
    answers = [parse_res(d, template=template) for d in data]
    accuracy = accuracy_score(gts, answers)
    print(accuracy)

In [33]:
get_accuracy(data, sivqa, template=0)

0.48828125


### eval yi-vl accuracy

In [4]:
data = []
with open(os.path.join(result_dir, "sivqa_yi-vl_prompt3.jsonl"), "r", encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

In [13]:
data[10:20]

[{'response': 'B', 'qid': 'vqa-13'},
 {'response': 'C) 咸鲜', 'qid': 'vqa-14'},
 {'response': '徽菜', 'qid': 'vqa-16'},
 {'response': 'C', 'qid': 'vqa-17'},
 {'response': 'C', 'qid': 'vqa-18'},
 {'response': '宁波', 'qid': 'vqa-19'},
 {'response': 'D', 'qid': 'vqa-21'},
 {'response': 'D', 'qid': 'vqa-22'},
 {'response': '(A) 湘菜', 'qid': 'vqa-23'},
 {'response': 'D', 'qid': 'vqa-24'}]

In [8]:

ans2idx = {
        "A":"0",
        "B":"1",
        "C":"2",
        "D":"3"
        }

def parse_res(res, template=0):
    ans_str = res["response"].strip()
    ans_letter = re.findall(r'[A-Z]', ans_str)
    if not ans_letter or len(ans_letter) == 0:
        print("can not parse ans for res: ", res)
        return random.choice(["0", "1", "2", "3"])
    else:
        ans = ans_letter[0].upper()
        if ans not in ans2idx:
            print("can not parse ans for res: ", res)
            return random.choice(["0", "1", "2", "3"])
        else:
            return ans2idx[ans]


In [12]:
get_accuracy(data, sivqa, template=3)

can not parse ans for res:  {'response': '京菜', 'qid': 'vqa-10'}
can not parse ans for res:  {'response': '徽菜', 'qid': 'vqa-16'}
can not parse ans for res:  {'response': '宁波', 'qid': 'vqa-19'}
can not parse ans for res:  {'response': '广东', 'qid': 'vqa-28'}
can not parse ans for res:  {'response': '客家', 'qid': 'vqa-34'}
can not parse ans for res:  {'response': '川渝', 'qid': 'vqa-36'}
can not parse ans for res:  {'response': '东北', 'qid': 'vqa-50'}
can not parse ans for res:  {'response': '南方', 'qid': 'vqa-57'}
can not parse ans for res:  {'response': '台州', 'qid': 'vqa-58'}
can not parse ans for res:  {'response': '贵州', 'qid': 'vqa-60'}
can not parse ans for res:  {'response': '粤菜', 'qid': 'vqa-61'}
can not parse ans for res:  {'response': '四川', 'qid': 'vqa-70'}
can not parse ans for res:  {'response': '川菜', 'qid': 'vqa-85'}
can not parse ans for res:  {'response': '粤', 'qid': 'vqa-92'}
can not parse ans for res:  {'response': '川菜', 'qid': 'vqa-93'}
can not parse ans for res:  {'response': 

In [ ]:
get_accuracy(data, sivqa, template=1)

0.48046875


In [7]:
get_accuracy(data, sivqa, template=2)

NameError: name 'get_accuracy' is not defined

In [18]:
get_accuracy(data, sivqa, template=3)

can not parse ans for res:  {'response': '新疆菜', 'qid': 'vqa-0'}
can not parse ans for res:  {'response': '川渝', 'qid': 'vqa-6'}
can not parse ans for res:  {'response': '香港', 'qid': 'vqa-8'}
can not parse ans for res:  {'response': '鲜甜', 'qid': 'vqa-9'}
can not parse ans for res:  {'response': '赣菜', 'qid': 'vqa-10'}
can not parse ans for res:  {'response': '氽', 'qid': 'vqa-13'}
can not parse ans for res:  {'response': '徽菜', 'qid': 'vqa-16'}
can not parse ans for res:  {'response': '香港', 'qid': 'vqa-18'}
can not parse ans for res:  {'response': '宁波', 'qid': 'vqa-19'}
can not parse ans for res:  {'response': '西北菜', 'qid': 'vqa-21'}
can not parse ans for res:  {'response': '北京', 'qid': 'vqa-22'}
can not parse ans for res:  {'response': '川菜', 'qid': 'vqa-24'}
can not parse ans for res:  {'response': '潮汕菜', 'qid': 'vqa-26'}
can not parse ans for res:  {'response': '客家', 'qid': 'vqa-28'}
can not parse ans for res:  {'response': '炒', 'qid': 'vqa-30'}
can not parse ans for res:  {'response': '客